In [7]:
try:
  # We must install required packages if we are in Google Colab
  import google.colab
  %pip install roboticstoolbox-python>=1.0.2
  %pip install qpsolvers[quadprog]
  COLAB = True
except:
    pass

# Numerical Inverse Kinematics Benchmark

<br>

In this Notebook we benchmark the performance of each numerical IK solver we provide with the toolbox.

There are several ways to use IK solvers with the Toolbox:

* The **fast** methods which are implemented in C++ and wrapped in Python. These methods are available from ``robot`` and ``ets`` classes and start with ``ik_``. For example ``ik_LM`` is a fast C++ based solver.

* The **slow** methods which are implemented in Python. These methods are available from ``robot`` and ``ets`` classes and start with ``ikine_``. For example ``ikine_LM`` is a slow Python based solver.

* The **slow** class-based operation which is implemented in Python. These classes inherit from the ``IKSolver`` class and start with ``IK_``. For example ``IK_LM`` is a class which provides a slow Python based solver.

### Contents

[1.0 Fast IK Methods](#1)

[2.0 Python IK Methods](#2)

[3.0 Python Class Based IK Methods](#3)



In [8]:
# We will do the imports required for this notebook here

# numpy provides import array and linear algebra utilities
import numpy as np

# the robotics toolbox provides robotics specific functionality
import roboticstoolbox as rtb

# ansitable is a great package for printing tables in a terminal
from ansitable import ANSITable

# a package for creating dynamic progress bars
from progress.bar import Bar

# swift is a lightweight browser-based simulator which comes with the toolbox
from swift import Swift

# spatialgeometry is a utility package for dealing with geometric objects
import spatialgeometry as sg

# provides sleep functionaly
import time

# suppress warnings
import warnings
warnings.filterwarnings('ignore')

<br>

<a id='1'></a>

### 1.0 Fast IK Methods
---

In [9]:
# Our robot and ETS
robot = rtb.models.Panda()
ets = robot.ets()

In [10]:
# Number of problems to solve
speed_problems = 10000

# Cartesion DoF priority matrix
mask = np.array([1.0, 1.0, 1.0, 1.0, 1.0, 1.0])

# random valid q values which will define Tep
q_rand = ets.random_q(speed_problems)

# Our desired end-effector poses
Tep = np.zeros((speed_problems, 4, 4))

for i in range(speed_problems):
    Tep[i] = ets.eval(q_rand[i])

# Maximum iterations allowed in a search
ilimit = 30

# Maximum searches allowed per problem
slimit = 100

# Solution tolerance
tol = 1e-6

joint_limits = False

In [11]:
speed_solvers = [
    lambda Tep: ets.ik_NR(
        Tep,
        ilimit=ilimit,
        slimit=slimit,
        tol=tol,
        mask=mask,
        pinv=True,
        joint_limits=joint_limits,
    ),
    lambda Tep: ets.ik_GN(
        Tep,
        ilimit=ilimit,
        slimit=slimit,
        tol=tol,
        mask=mask,
        pinv=True,
        joint_limits=joint_limits,
    ),
    lambda Tep: ets.ik_LM(
        Tep, ilimit=ilimit, slimit=slimit, tol=tol, mask=mask, k=1e-4, joint_limits=joint_limits, method="wampler"
    ),
    lambda Tep: ets.ik_LM(
        Tep,
        q0=None,
        ilimit=ilimit,
        slimit=slimit,
        tol=tol,
        mask=mask,
        k=0.1,
        joint_limits=joint_limits,
        method="chan",
    ),
    lambda Tep: ets.ik_LM(
        Tep, ilimit=ilimit, slimit=slimit, tol=tol, mask=mask, k=0.0001, joint_limits=joint_limits, method="sugihara"
    ),
]

speed_names = [
    "Newton Raphson (pinv=True)",
    "Gauss Newton (pinv=True)",
    "LM Wampler 1e-4",
    "LM Chan 0.1",
    "LM Sugihara 0.0001",
]

times = []

In [12]:
for name, solver in zip(speed_names, speed_solvers):
    print(f"Solving with {name}")

    start = time.time()

    for i in range(speed_problems):
        solver(Tep[i])

    total_time = time.time() - start
    times.append(total_time)


print(f"\nNumerical Inverse Kinematics Methods Times Compared over {speed_problems} problems\n")

table = ANSITable(
    "Method",
    "Total Time (s)",
    "Average Time per Solution (μs)",
    border="thin",
)

for name, t in zip(speed_names, times):
    table.row(
        name,
        np.round(t, 4),
        np.round((t / speed_problems) * 1e6, 4),
    )

table.print()

Solving with Newton Raphson (pinv=True)
Solving with Gauss Newton (pinv=True)
Solving with LM Wampler 1e-4
Solving with LM Chan 0.1
Solving with LM Sugihara 0.0001

Numerical Inverse Kinematics Methods Times Compared over 10000 problems

┌───────────────────────────┬────────────────┬────────────────────────────────┐
│                    Method │ Total Time (s) │ Average Time per Solution (μs) │
├───────────────────────────┼────────────────┼────────────────────────────────┤
│Newton Raphson (pinv=True) │         1.8708 │                       187.0795 │
│  Gauss Newton (pinv=True) │          2.474 │                       247.3969 │
│           LM Wampler 1e-4 │         0.6079 │                        60.7928 │
│               LM Chan 0.1 │         0.2724 │                        27.2353 │
│        LM Sugihara 0.0001 │         0.3546 │                        35.4575 │
└───────────────────────────┴────────────────┴────────────────────────────────┘

